In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import requests # from urllib.request import urlopen

from bs4 import BeautifulSoup 
import time
from datetime import datetime
import re
import pandas as pd
import numpy as np

## Flix Petrol website <- get country name + streaming name + date

In [2]:
url = "https://flixpatrol.com/top10/netflix" 
web = requests.get(url).content
source = BeautifulSoup(web, 'html.parser')

In [3]:
list_dropdown = []
for dropdown in source.find_all('a', {'class' : "dropdown-item"}):
    dropdown = dropdown["href"][7:]
    list_dropdown.append(dropdown)

print(list_dropdown)

['streaming', 'amazon', 'disney', 'fandango', 'google', 'hbo', 'hulu', 'imdb', 'itunes', 'netflix', 'vudu', 'netflix/world', 'netflix/argentina', 'netflix/australia', 'netflix/austria', 'netflix/bangladesh', 'netflix/belgium', 'netflix/bolivia', 'netflix/brazil', 'netflix/bulgaria', 'netflix/canada', 'netflix/colombia', 'netflix/costa-rica', 'netflix/croatia', 'netflix/cyprus', 'netflix/czech-republic', 'netflix/denmark', 'netflix/dominican-republic', 'netflix/ecuador', 'netflix/egypt', 'netflix/estonia', 'netflix/finland', 'netflix/france', 'netflix/germany', 'netflix/greece', 'netflix/guatemala', 'netflix/honduras', 'netflix/hong-kong', 'netflix/hungary', 'netflix/chile', 'netflix/iceland', 'netflix/india', 'netflix/ireland', 'netflix/israel', 'netflix/italy', 'netflix/japan', 'netflix/jordan', 'netflix/kuwait', 'netflix/latvia', 'netflix/lebanon', 'netflix/lithuania', 'netflix/luxembourg', 'netflix/malaysia', 'netflix/mexico', 'netflix/morocco', 'netflix/netherlands', 'netflix/new-z

In [4]:
#Streaming service type
streaming_list=list_dropdown[1:11]
print(streaming_list)

['amazon', 'disney', 'fandango', 'google', 'hbo', 'hulu', 'imdb', 'itunes', 'netflix', 'vudu']


In [5]:
#Country List

country_list=list_dropdown[11:-13]

final_country=[]
for item in country_list:
    final_country.append(item[8:])
    
print(final_country)

['world', 'argentina', 'australia', 'austria', 'bangladesh', 'belgium', 'bolivia', 'brazil', 'bulgaria', 'canada', 'colombia', 'costa-rica', 'croatia', 'cyprus', 'czech-republic', 'denmark', 'dominican-republic', 'ecuador', 'egypt', 'estonia', 'finland', 'france', 'germany', 'greece', 'guatemala', 'honduras', 'hong-kong', 'hungary', 'chile', 'iceland', 'india', 'ireland', 'israel', 'italy', 'japan', 'jordan', 'kuwait', 'latvia', 'lebanon', 'lithuania', 'luxembourg', 'malaysia', 'mexico', 'morocco', 'netherlands', 'new-zealand', 'nicaragua', 'nigeria', 'norway', 'oman', 'pakistan', 'panama', 'paraguay', 'peru', 'philippines', 'poland', 'portugal', 'qatar', 'romania', 'russia', 'saudi-arabia', 'serbia', 'singapore', 'slovakia', 'slovenia', 'south-africa', 'south-korea', 'spain', 'sri-lanka', 'sweden', 'switzerland', 'taiwan', 'thailand', 'turkey', 'ukraine', 'united-arab-emirates', 'united-kingdom', 'united-states', 'uruguay', 'venezuela', 'vietnam']


In [6]:
#Time list
time_list=list_dropdown[-13:-6]

final_time=[]
for item in time_list:
    final_time.append(item[16:])
    
print(final_time)

['20-12-28', '20-12-27', '20-053', '20-052', '20-12', '20-11', '20']


## Crawling 

In [25]:
def main_crawling(streaming, country_name, date):
    url = "https://flixpatrol.com/top10/" + streaming + "/" + country_name + "/" + date


    web_ranking = requests.get(url).content
    source_ranking = BeautifulSoup(web_ranking, 'html.parser')

    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    web_ranking = requests.get(url, headers=headers).content

    titles = []
    points = []

    movies = []
    tv_show = []
    points_movies = []
    points_tv_show = []

    country = [country_name.capitalize()] * 10
    rank = list(range(1,11))


    #Titles
    if source_ranking.find_all('div', {'class' : 'threedots'}) == []:
        titles = ['-'] * 20
    else:
        for title in source_ranking.find_all('div', {'class' : 'threedots'})[:20]: 
            top10 = title.find('a').img['alt']
            titles.append(top10)

    #Points <-수정
    if source_ranking.find_all('div', {'class' : 'col-lg-1 col-2 pt-1 text-right'}) == []:
        points = ['-'] * 20
    else:
        for item in source_ranking.find_all('div', {'class' : 'col-lg-1 col-2 pt-1 text-right'})[:20]:
            point = item.get_text()
            point = point.replace('\xa0','')
            point = int(point)
            points.append(point)

    #Divide into 2 categories: Movies / TV Show
    movies = titles[:10]
    tv_show = titles[10:]
    points_movies = points[:10]
    points_tv_show = points[10:]

        ### Additional info about country
    try:
        #Region
        region = source_ranking.find_all('div', {'class' : 'col-4'})[0].get_text()

        #Population
        population = source_ranking.find_all('div', {'class' : 'col-4'})[1].get_text().split()
        population = ''.join(population)
        population = int(population)

        #GDP per capita
        gdp_per_capita = source_ranking.find_all('div', {'class' : 'col-4'})[2].get_text().split()
        gdp_per_capita = ''.join(gdp_per_capita)
        gdp_per_capita = int(gdp_per_capita[1:])

        #########################

        #Netflix launched on
        netflix_launched = source_ranking.find_all('div', {'class' : 'col-5'})[0].get_text()

        #Netflix titles
        netflix_titles = source_ranking.find_all('div', {'class' : 'col-5'})[1].get_text().split()
        netflix_titles = ''.join(netflix_titles)
        netflix_titles = int(netflix_titles[:-1])

        #Netflix subscribers
        netflix_subs = source_ranking.find_all('div', {'class' : 'col-5'})[2].get_text().split()
        netflix_subs = ''.join(netflix_subs)
        netflix_subs = int(netflix_subs[:-1])

    except:
        region = '-'
        population = '-'
        gdp_per_capita = '-'
        netflix_launched = '-'
        netflix_titles = '-'
        netflix_subs= '-'
    
########################################################################

    region =[region] * 10
    population = [population] * 10
    gdp_per_capita = [gdp_per_capita] * 10
    netflix_launched = [netflix_launched] * 10
    netflix_titles =[netflix_titles] * 10 
    netflix_subs = [netflix_subs] * 10

    ranking_df = pd.DataFrame({'Movies':movies, 
                                'Movie Point':points_movies, 
                                'TV shows':tv_show, 
                                'TV show Point':points_tv_show,
                                'Country': country,
                                'Ranking': rank,
                                'Region': region,
                                'Population': population,
                                'GDP per capita': gdp_per_capita,
                                'Netflix Launch Date': netflix_launched,
                                'Netflix Titles': netflix_titles,
                                'Netflix Subscribers': netflix_subs})
        
    ranking_df.to_excel('flixpetrol_{0}_{1}_{2}.xlsx'.format(streaming, country_name, date), index=False, encoding='utf-8')
    
    
    print('\n크롤링이 성공적으로 완료되었습니다!')
    print('\n{0}/{1} 크롤링 결과를 다음 파일에 저장하였습니다'.format(streaming, country_name))    
        

In [26]:
main_crawling(streaming='netflix', country_name='luxembourg', date='2020')


크롤링이 성공적으로 완료되었습니다!

netflix/luxembourg 크롤링 결과를 다음 파일에 저장하였습니다


In [209]:
#Try one country
streaming = input('Type streaming service : ')
country_name = input('Type name of country : ')
date = input('Type date : ')

Type streaming service : netflix
Type name of country : austria
Type date : 2020


In [212]:
main_crawling(streaming, country_name, date)


크롤링이 성공적으로 완료되었습니다!

netflix/austria/<module 'time' (built-in)> 크롤링 결과를 다음 파일에 저장하였습니다


## Final crawling for all countries in 2020 (Netflix)

In [21]:
for item in final_country:
    main_crawling(streaming = 'netflix', country_name = item, date='2020')


크롤링이 성공적으로 완료되었습니다!

netflix/world 크롤링 결과를 다음 파일에 저장하였습니다

크롤링이 성공적으로 완료되었습니다!

netflix/argentina 크롤링 결과를 다음 파일에 저장하였습니다

크롤링이 성공적으로 완료되었습니다!

netflix/australia 크롤링 결과를 다음 파일에 저장하였습니다

크롤링이 성공적으로 완료되었습니다!

netflix/austria 크롤링 결과를 다음 파일에 저장하였습니다

크롤링이 성공적으로 완료되었습니다!

netflix/bangladesh 크롤링 결과를 다음 파일에 저장하였습니다

크롤링이 성공적으로 완료되었습니다!

netflix/belgium 크롤링 결과를 다음 파일에 저장하였습니다

크롤링이 성공적으로 완료되었습니다!

netflix/bolivia 크롤링 결과를 다음 파일에 저장하였습니다

크롤링이 성공적으로 완료되었습니다!

netflix/brazil 크롤링 결과를 다음 파일에 저장하였습니다

크롤링이 성공적으로 완료되었습니다!

netflix/bulgaria 크롤링 결과를 다음 파일에 저장하였습니다

크롤링이 성공적으로 완료되었습니다!

netflix/canada 크롤링 결과를 다음 파일에 저장하였습니다

크롤링이 성공적으로 완료되었습니다!

netflix/colombia 크롤링 결과를 다음 파일에 저장하였습니다

크롤링이 성공적으로 완료되었습니다!

netflix/costa-rica 크롤링 결과를 다음 파일에 저장하였습니다

크롤링이 성공적으로 완료되었습니다!

netflix/croatia 크롤링 결과를 다음 파일에 저장하였습니다

크롤링이 성공적으로 완료되었습니다!

netflix/cyprus 크롤링 결과를 다음 파일에 저장하였습니다

크롤링이 성공적으로 완료되었습니다!

netflix/czech-republic 크롤링 결과를 다음 파일에 저장하였습니다

크롤링이 성공적으로 완료되었습니다!

netflix/denmark 크롤링 결과를 다음 파일에 저장하였습니다

크롤링이 성공적으

In [257]:
# #Delete countries with error
# final_country.remove('world')
# final_country.remove('bangladesh')
# final_country.remove('honduras')
# final_country.remove('jordan')
# final_country.remove('lebanon')
# final_country.remove('luxembourg')
# final_country.remove('nicaragua')
# final_country.remove('oman')
# final_country.remove('pakistan')
# final_country.remove('panama')
# final_country.remove('paraguay')
# final_country.remove('serbia')
# final_country.remove('sri-lanka')
# final_country.remove('united-arab-emirates')
# final_country.remove('uruguay')
# final_country.remove('vietnam')

In [22]:
##Concat all countries in one dataframe 

#Read excel file
chunk =[]

for i in final_country:
    df=pd.read_excel('flixpetrol_netflix_{}_2020.xlsx'.format(i), index=False, encoding='utf-8')
    chunk.append(df)
    merged_country_netflix_2020 = pd.concat(chunk, ignore_index=True)

merged_country_netflix_2020.head(3)
merged_country_netflix_2020.to_excel('merged_country_netflix_2020.xlsx', index=False, encoding='utf-8')

In [23]:
#Len must be equal!
print(len(merged_country_netflix_2020), len(final_country)*10)

810 810


In [14]:
merged_country_netflix_2020=pd.read_excel('merged_country_netflix_2020.xlsx', index=False, encoding='utf-8')
df=merged_country_netflix_2020
df.head()

,Movies,Movie Point,TV shows,TV show Point,Country,Ranking,Region,Population,GDP per capita,Netflix Launch Date,Netflix Titles,Netflix Subscribers
0,365 Days,42026,The Queen's Gambit,42701,World,1,-,-,-,-,-,-
1,Enola Holmes,16432,Emily in Paris,27136,World,2,-,-,-,-,-,-
2,The Christmas Chronicles: Part Two,15183,Lucifer,25969,World,3,-,-,-,-,-,-
3,Holidate,14684,The Umbrella Academy,23427,World,4,-,-,-,-,-,-
4,The Old Guard,13908,Money Heist,22491,World,5,-,-,-,-,-,-


In [18]:
len(df['Movies'].unique())

148

In [27]:
grouped=df["Movie Point"].groupby(df["Movies"])
point_bymovie=grouped.sum()
df_point_bymovie = point_bymovie.to_frame()
df_point_bymovie

,Movie Point
Movies,
#Alive,2400
-,----------------------------------------------...
12 Strong,341
365 Days,81279
5Gang: A different kind of Christmas,214
6 Underground,3426
A Million Ways to Die in the West,570
A Whisker Away,255
After We Collided,156


In [22]:
pd.pivot_table(df, index='Movies', aggfunc=np.sum)

,Ranking
Movies,
#Alive,90
-,385
12 Strong,20
365 Days,102
5Gang: A different kind of Christmas,4
6 Underground,25
A Million Ways to Die in the West,24
A Whisker Away,4
After We Collided,9


In [ ]:
###### practice ######

In [48]:
# Example: https://flixpatrol.com/top10/netflix/argentina/2020-11-23
url = "https://flixpatrol.com/top10"
url_list=[]

for streaming in streaming_list:
    for country in final_country:
        for date in final_time:
             url_list.append("https://flixpatrol.com/top10/{0}/{1}/{2}".format(streaming, country, date))

print(url_list)

['https://flixpatrol.com/top10/amazon/world/2020-12-24', 'https://flixpatrol.com/top10/amazon/world/2020-12-23', 'https://flixpatrol.com/top10/amazon/world/2020-052', 'https://flixpatrol.com/top10/amazon/world/2020-051', 'https://flixpatrol.com/top10/amazon/world/2020-12', 'https://flixpatrol.com/top10/amazon/world/2020-11', 'https://flixpatrol.com/top10/amazon/world/2020', 'https://flixpatrol.com/top10/amazon/albania/2020-12-24', 'https://flixpatrol.com/top10/amazon/albania/2020-12-23', 'https://flixpatrol.com/top10/amazon/albania/2020-052', 'https://flixpatrol.com/top10/amazon/albania/2020-051', 'https://flixpatrol.com/top10/amazon/albania/2020-12', 'https://flixpatrol.com/top10/amazon/albania/2020-11', 'https://flixpatrol.com/top10/amazon/albania/2020', 'https://flixpatrol.com/top10/amazon/argentina/2020-12-24', 'https://flixpatrol.com/top10/amazon/argentina/2020-12-23', 'https://flixpatrol.com/top10/amazon/argentina/2020-052', 'https://flixpatrol.com/top10/amazon/argentina/2020-051

In [23]:
#Top 10: Argentina 2020 - Try 

url = "https://flixpatrol.com/top10/netflix/argentina/2020" 
web_news = requests.get(url).content
source_news = BeautifulSoup(web_news, 'html.parser')

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
web_news = requests.get(url, headers=headers).content

#for idx, source in enumerate(source_news.find_all('div', {'class' : 'col-lg-6 col-6 pt-1 pb-1 threedots'})[:11],1):
#    print ("Rank {0}: {1}".format(idx, source.get_text()[:11]))

titles = []
points = []

movies = []
tv_show = []
points_movies = []
points_tv_show = []

for item in source_news.find_all('div', {'class' : 'col-lg-6 col-6 pt-1 pb-1 threedots'}):
    title = item.get_text().strip()
    titles.append(title)
    print(title)
    
for item in source_news.find_all('div', {'class' : 'col-lg-1 col-2 pt-1 text-right'}):
    point = item.get_text().strip()
    points.append(point)
    print(point)

#Divide into 2 categories: Movies / TV Show
movies = titles[:10]
tv_show = titles[10:]
points_movies = points[:10]
points_tv_show = points[10:]

rank_df = pd.DataFrame({'Movies':movies, 
                           'Movie Point':points_movies, 
                           'TV shows':tv_show, 
                           'TV show Point':points_tv_show})

#rank_df.to_excel('flixpetrol_{0}_{1}_{2}.xlsx'.format(streaming, country, date), index=False, encoding='utf-8')
rank_df.head()


365 Days
Wonder
Over the Moon
The Christmas Chronicles: Part Two
Enola Holmes
Perdida
The Emoji Movie
The SpongeBob Movie: Sponge on the ...
American Murder: The Family Next Do...
So Much Love to Give
Amar y Vivir
The Queen's Gambit
100 Dias Para Enamorarnos
Her Mother's Killer
Dark
Dark Desire
Cobra Kai
Locked Up
The Last Dance
The Crown
366
222
213
212
198
194
191
167
157
146
574
535
491
490
436
363
340
317
278
254


,Movies,Movie Point,TV shows,TV show Point
0,365 Days,366,Amar y Vivir,574
1,Wonder,222,The Queen's Gambit,535
2,Over the Moon,213,100 Dias Para Enamorarnos,491
3,The Christmas Chronicles: Part Two,212,Her Mother's Killer,490
4,Enola Holmes,198,Dark,436


In [22]:
url = "https://flixpatrol.com/top10/netflix/honduras/2020" 
web_news = requests.get(url).content
source_news = BeautifulSoup(web_news, 'html.parser')

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
web_news = requests.get(url, headers=headers).content

In [20]:
source_news.find('div', {'class' : 'threedots'}).find('img').attrs['alt']

'365 Days'

In [21]:
for news in source_news.find_all('div', {'class' : 'threedots'}):
    title = news.find('img').attrs['alt']
    print(title)
    #title = a.find('a').get('href')[7:]
    

#for title in titles: col-lg-6 col-6 pt-1 pb-1 threedots
#    print(title.find('a').get_text().strip())

365 Days
Wonder
The Christmas Chronicles: Part Two
Over the Moon
Enola Holmes
Perdida
The Emoji Movie
The SpongeBob Movie: Sponge on the Run
American Murder: The Family Next Door
So Much Love to Give
Amar y Vivir
The Queen's Gambit
100 Dias Para Enamorarnos
Her Mother's Killer
Dark
Dark Desire
Cobra Kai
Locked Up
The Last Dance
The Crown


In [150]:
source_news.find_all('div', {'class' : 'threedots'})

[<div class="col-lg-6 col-6 pt-1 pb-1 threedots">
 <a href="/title/365-days"><img alt="365 Days" class="img-border float-left mr-3 mr-md-5" src="https://filmtoro.cz/img3/film/im0jKiVtVyxynKVnzOyd6efbqYE.jpg" title="365 Days" width="30"/>365 Days</a>
 </div>, <div class="col-lg-6 col-6 pt-1 pb-1 threedots">
 <a href="/title/wonder"><img alt="Wonder" class="img-border float-left mr-3 mr-md-5" src="https://filmtoro.cz/img3/film/ouYgAatYH4JzIThj6FI3UYf31RI.jpg" title="Wonder" width="30"/>Wonder</a>
 </div>, <div class="col-lg-6 col-6 pt-1 pb-1 threedots">
 <a href="/title/over-the-moon-2020"><img alt="Over the Moon" class="img-border float-left mr-3 mr-md-5" src="https://filmtoro.cz/img3/film/lQfdytwN7eh0tXWjIiMceFdBBvD.jpg" title="Over the Moon" width="30"/>Over the Moon</a>
 <span class="badge badge-netflix ml-1" data-placement="left" data-toggle="tooltip" title="This title is Netflix original"> </span>
 </div>, <div class="col-lg-6 col-6 pt-1 pb-1 threedots">
 <a href="/title/the-christ

In [199]:
img = source_news.find('div', {'class' : 'threedots'}).find('img', alt=True)
img['alt']

'365 Days'

In [134]:
titles = source_news.find_all('div', {'class' : 'threedots'})

for idx, title in enumerate(titles):
    text = title.find('a').get_text('img alt')
    print(idx, text)

0 365 Days
1 Wonder
2 Over the Moon
3 The Christmas Chronicles: Part Two
4 Enola Holmes
5 Perdida
6 The Emoji Movie
7 The SpongeBob Movie: Sponge on the ...
8 American Murder: The Family Next Do...
9 So Much Love to Give
10 Amar y Vivir
11 The Queen's Gambit
12 100 Dias Para Enamorarnos
13 Her Mother's Killer
14 Dark
15 Dark Desire
16 Cobra Kai
17 Locked Up
18 The Last Dance
19 The Crown


In [99]:
title_list = source_news.find_all('div', {'class' : 'threedots'})


for idx, title in enumerate(title_list):
    try:
        if(idx%10 == 0):
            print("\n")
            
        text = title.find('a')
        print(idx, ":", text.get_text().strip())
    
    except:
        break



0 : 365 Days
1 : Wonder
2 : Over the Moon
3 : The Christmas Chronicles: Part Two
4 : Enola Holmes
5 : Perdida
6 : The Emoji Movie
7 : The SpongeBob Movie: Sponge on the ...
8 : American Murder: The Family Next Do...
9 : So Much Love to Give


10 : Amar y Vivir
11 : The Queen's Gambit
12 : 100 Dias Para Enamorarnos
13 : Her Mother's Killer
14 : Dark
15 : Dark Desire
16 : Cobra Kai
17 : Locked Up
18 : The Last Dance
19 : The Crown


In [47]:
for item in source_news.find_all('div', {'class' : 'col-4'}):
    print(item.get_text())


Latin America
44 938 712
$ 20 055


In [56]:
#Region
region = source_news.find_all('div', {'class' : 'col-4'})[0].get_text()

'Latin America'

In [62]:
#Population
population = source_news.find_all('div', {'class' : 'col-4'})[1].get_text().split()
population = ''.join(population)
population = int(population)
population

44938712

In [64]:
#GDP per capita
gdp_per_capita = source_news.find_all('div', {'class' : 'col-4'})[2].get_text().split()
gdp_per_capita = ''.join(gdp_per_capita)
gdp_per_capita = int(gdp_per_capita[1:])
gdp_per_capita

20055

In [46]:
#Netflix launched on
netflix_launched = source_news.find_all('div', {'class' : 'col-5'})[0].get_text() 



if netflix_launched == '' :
    netflix_launched = '-'
    
netflix_launched = [netflix_launched]*10
netflix_launched



['-', '-', '-', '-', '-', '-', '-', '-', '-', '-']

In [74]:
netflix_titles = source_news.find_all('div', {'class' : 'col-5'})[1].get_text().split()
netflix_titles=''.join(netflix_titles)
netflix_titles

'3000+'

In [71]:
if netflix_subs == [] :
    netflix_subs = '-'
else:
    netflix_subs = ''.join(netflix_subs)
    netflix_subs = int(netflix_subs[:-1])

## Testing station

In [174]:
url = "https://flixpatrol.com/top10/netflix/world/2020" 
web_news = requests.get(url).content
source_news = BeautifulSoup(web_news, 'html.parser')

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
web_news = requests.get(url, headers=headers).content

In [87]:
titles=[]
if source_news.find_all('div', {'class' : 'col-lg-1 col-2 pt-1 text-right'}) == []:
    titles = ['-'] * 10
else:
    for title in source_news.find_all('div', {'class' : 'threedots'}): 
        top10 = title.find('a').img['alt']
        titles.append(top10)

In [29]:
url = "https://flixpatrol.com/top10/netflix/argentina/2020" 
web_news = requests.get(url).content
source_news = BeautifulSoup(web_news, 'html.parser')

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
web_news = requests.get(url, headers=headers).content

In [30]:
country_name='argentina'

titles = []
points = []

movies = []
tv_show = []
points_movies = []
points_tv_show = []

country = [country_name.capitalize()] * 10
rank = list(range(1,11))


try: 
    #Titles
    for title in source_news.find_all('div', {'class' : 'threedots'})[:20]: 
            top10 = title.find('a').img['alt']
            titles.append(top10)

    #Points <-수정
    for item in source_news.find_all('div', {'class' : 'col-lg-1 col-2 pt-1 text-right'})[:20]:
            point = item.get_text()
            point = point.replace('\xa0','')
            point = int(point)
            points.append(point)

        #print(point)


    #Divide into 2 categories: Movies / TV Show
    movies = titles[:10]
    tv_show = titles[10:]
    points_movies = points[:10]
    points_tv_show = points[10:]

    ### Additional info about country

    #Region
    region = source_news.find_all('div', {'class' : 'col-4'})[0].get_text()

    #Population
    population = source_news.find_all('div', {'class' : 'col-4'})[1].get_text().split()
    population = ''.join(population)
    population = int(population)

    #GDP per capita
    gdp_per_capita = source_news.find_all('div', {'class' : 'col-4'})[2].get_text().split()
    gdp_per_capita = ''.join(gdp_per_capita)
    gdp_per_capita = int(gdp_per_capita[1:])
    
    #########################

    #Netflix launched on
    netflix_launched = source_news.find_all('div', {'class' : 'col-5'})[0].get_text()

    #Netflix titles
    netflix_titles = source_news.find_all('div', {'class' : 'col-5'})[1].get_text().split()
    netflix_titles = ''.join(netflix_titles)
    netflix_titles = int(netflix_titles[:-1])

    #Netflix subscribers
    netflix_subs = source_news.find_all('div', {'class' : 'col-5'})[2].get_text().split()
    netflix_subs = ''.join(netflix_subs)
    netflix_subs = int(netflix_subs[:-1])
    
except:
    titles = ['-'] * 20
    points = ['-'] * 20

    region = '-'
    population = '-'
    gdp_per_capita = '-'
    netflix_launched = '-'
    netflix_titles = '-'
    netflix_subs= '-'
    
############

region =[region] * 10
population = [population] * 10
gdp_per_capita = [gdp_per_capita] * 10
netflix_launched = [netflix_launched] * 10
netflix_titles =[netflix_titles] * 10 
netflix_subs = [netflix_subs] * 10

ranking_df = pd.DataFrame({'Movies':movies, 
                            'Movie Point':points_movies, 
                            'TV shows':tv_show, 
                            'TV show Point':points_tv_show,
                            'Country': country,
                            'Ranking': rank,
                            'Region': region,
                            'Population': population,
                            'GDP per capita': gdp_per_capita,
                            'Netflix Launch Date': netflix_launched,
                            'Netflix Titles': netflix_titles,
                            'Netflix Subscribers': netflix_subs})
ranking_df.head()

,Movies,Movie Point,TV shows,TV show Point,Country,Ranking,Region,Population,GDP per capita,Netflix Launch Date,Netflix Titles,Netflix Subscribers
0,365 Days,366,Amar y Vivir,574,Argentina,1,Latin America,44938712,20055,2011-09-07,4400,4000000
1,Wonder,222,The Queen's Gambit,542,Argentina,2,Latin America,44938712,20055,2011-09-07,4400,4000000
2,The Christmas Chronicles: Part Two,214,100 Dias Para Enamorarnos,491,Argentina,3,Latin America,44938712,20055,2011-09-07,4400,4000000
3,Over the Moon,213,Her Mother's Killer,490,Argentina,4,Latin America,44938712,20055,2011-09-07,4400,4000000
4,Enola Holmes,198,Dark,436,Argentina,5,Latin America,44938712,20055,2011-09-07,4400,4000000


In [103]:
print(tv_show,points_tv_show,netflix_subs)

[] [] 150000


In [30]:
source_news.find_all('div', {'class' : 'col-5'})[1].get_text()

'3000+'

In [66]:
#Netflix titles
netflix_titles = source_news.find_all('div', {'class' : 'col-5'})[1].get_text().split()
netflix_titles = ''.join(netflix_titles)
netflix_titles = int(netflix_titles[:-1])
netflix_titles

4400

In [67]:
#Netflix subscribers
netflix_subs = source_news.find_all('div', {'class' : 'col-5'})[2].get_text().split()
netflix_subs = ''.join(netflix_subs)
netflix_subs = int(netflix_subs[:-1])
netflix_subs

4000000

In [25]:
#Get points for each show

for item in source_news.find_all('div', {'class' : 'col-lg-1 col-2 pt-1 text-right'}):
    item = item.get_text()
    item = item.replace('\xa0','')
    item = int(item)
    print(item)


361
247
242
217
202
200
195
178
159
158
1197
1037
823
814
708
452
408
393
381
361
